In [1]:
%%bash
mkdir -p ../datasets/arem
cd ../datasets/arem
rm AReM.zip
wget https://archive.ics.uci.edu/ml/machine-learning-databases/00366/AReM.zip
unzip AReM.zip

Archive:  AReM.zip
   creating: bending1/
  inflating: bending1/dataset1.csv   
  inflating: bending1/dataset2.csv   
  inflating: bending1/dataset3.csv   
  inflating: bending1/dataset4.csv   
  inflating: bending1/dataset5.csv   
  inflating: bending1/dataset6.csv   
  inflating: bending1/dataset7.csv   
   creating: bending2/
  inflating: bending2/dataset1.csv   
  inflating: bending2/dataset2.csv   
  inflating: bending2/dataset3.csv   
  inflating: bending2/dataset4.csv   
  inflating: bending2/dataset5.csv   
  inflating: bending2/dataset6.csv   
  inflating: bendingType.pdf         
   creating: cycling/
  inflating: cycling/dataset1.csv    
  inflating: cycling/dataset10.csv   
  inflating: cycling/dataset11.csv   
  inflating: cycling/dataset12.csv   
  inflating: cycling/dataset13.csv   
  inflating: cycling/dataset14.csv   
  inflating: cycling/dataset15.csv   
  inflating: cycling/dataset2.csv    
  inflating: cycling/dataset3.csv    
  inflating: cycling/dataset4.csv    
 

rm: cannot remove 'AReM.zip': No such file or directory
--2019-08-09 23:48:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/00366/AReM.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 524200 (512K) [application/x-httpd-php]
Saving to: ‘AReM.zip’

     0K .......... .......... .......... .......... ..........  9%  104K 4s
    50K .......... .......... .......... .......... .......... 19%  316K 3s
   100K .......... .......... .......... .......... .......... 29% 13.2M 2s
   150K .......... .......... .......... .......... .......... 39%  311K 1s
   200K .......... .......... .......... .......... .......... 48% 31.8M 1s
   250K .......... .......... .......... .......... .......... 58%  332K 1s
   300K .......... .......... .......... .......... .......... 68% 12.1M 0s
   350K .......... .......... ......

In [2]:
import os
from random import Random
import pandas as pd

In [3]:
dataset_dir = '../datasets/arem/'

def get_activity_files(activity):
    activity_dir = os.path.join(dataset_dir, activity)
    activity_files = [ os.path.join(activity_dir, file) for file in sorted(os.listdir(activity_dir)) ]
    return sorted(activity_files)

target_activities = ['cycling', 'lying', 'sitting', 'standing', 'walking']

all_activity_files = [ (activity, file) for activity in target_activities for file in get_activity_files(activity) ]

shuffled_activity_files = all_activity_files.copy()
Random(1).shuffle(shuffled_activity_files)

In [4]:
def load_df(activity, file):
    df = pd.read_csv(file, names=['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'], comment='#')
    df = df.drop(['time'], axis=1)
    df['activity'] = activity
    return df

case_dfs = [ load_df(activity, file) for activity, file in shuffled_activity_files ]

In [5]:
def random_interleave_dfs(dfs, seed=1):
    rand = Random(seed)
    if not dfs:
        return None
    all_df = pd.DataFrame(columns = dfs[0].columns)
    dfs = [ df.copy() for df in dfs if not df.empty ]
    # While there are still non-empty dataframes
    while dfs:
        next_df = rand.choice(dfs)
        # Append first row from a random df
        all_df = all_df.append(next_df.iloc[0], ignore_index=True)
        # Remove that row from the random df
        next_df.drop(next_df.index[0], inplace=True)
        # Remove any empty dataframes
        dfs = [ df for df in dfs if not df.empty ]
    return all_df
all_cases_interleaved_df = random_interleave_dfs(case_dfs)

In [6]:
all_cases_interleaved_df.to_csv(os.path.join(dataset_dir, 'arem_all_interleaved.csv'), index=False, na_rep='NaN')